### **5.3 - Local Executor con with PostgreSQL**

#### **`parallelism`**
#### **`dag_concurrency`**
#### **`max_active_runs_per_dag`**
#### **`max_active_runs`**

En el vídeo anterior hemos visto lo que es el Sequential Executor con SQLite. Esta configuración es la que se obtiene por defecto después de haber instalado Airflow, y es perfecta para pruebas y debugging. Ahora, supongamos que quieres pasar a producción con Airflow, ¿qué ejecutor debes utilizar? Bueno, antes de configurar Airflow con el Celery Executor o el Kubernetes Executor, que son mucho más complejos de manejar, deberías probar el Local Executor. Vamos a descubrirlo. 

<center><img src="https://i.postimg.cc/NMnVQ6P2/a751.png"></center>

El Local Executor te permite ejecutar tareas en paralelo, que se ejecutan en una sola máquina como tu laptop o servidor. Para ello, se basa en el módulo multiproceso de Python donde cada vez que una tarea necesita ser disparada se genera un proceso. Por ejemplo, si un DAG dado tiene cinco tareas que ejecutar, entonces el Local Executor generará cinco procesos, un proceso por cada tarea, para ejecutarlas en paralelo. Más adelante veremos que podemos limitar el número de procesos modificando el archivo de configuración de Airflow. Las ventajas de utilizar el Local Executor son las siguientes: es muy sencillo y fácil de configurar. No hay que instalar otros componentes, todo se ejecuta en la misma máquina en la que se ejecuta Airflow. La generación de procesos es relativamente barata y consume pocos recursos, al tiempo que ofrece una forma robusta de utilizar el paralelismo sin los problemas que surgen cuando se empieza a distribuir la carga de trabajo entre muchas máquinas en una arquitectura de nodos múltiples. Obviamente, el último punto también es un inconveniente, ya que implica un único punto de fallo: si la máquina deja de funcionar, Airflow deja de funcionar y las tareas también. Además, el número de procesos, y por tanto el número de tareas que podrá ejecutar en paralelo, depende estrictamente de los recursos disponibles en la máquina. Cuantas más tareas quieras ejecutar, más recursos tendrás que añadir en términos de CPU, memoria, etc. Es el inconveniente del escalado vertical. No obstante, como mejor práctica, se recomienda encarecidamente comenzar con el Local Executor. Dado que Airflow es un orquestador, no debería tener grandes cargas de trabajo (workloads) que realizar y, por lo tanto, ejecutar sus tareas con el Local Executor puede ser más que suficiente para gestionar sus DAGs. Al final, depende de ti hacer tus benchmarks y pruebas para comprobar si el Local Executor se ajusta o no a tus necesidades. Trata de mantener las cosas simples. Dicho esto, sigamos adelante. 

<center><img src="https://i.postimg.cc/MHjLHnhP/a752.png"></center>

Para usar el Local Executor tienes que elegir una base de datos diferente a SQLite ya que no soporta escrituras concurrentes. En nuestro caso vamos a usar PostgreSQL. PostgreSQL es una base de datos objeto-relacional de código abierto con un enfoque particular en la extensibilidad y el cumplimiento de estándares. Es una base de datos cliente-servidor, lo que significa que hay un servidor que gestiona las conexiones, archivos y demás, así como el cliente que realiza las operaciones de la base de datos. PostgreSQL es compatible con ACID y admite funciones SQL estándar y avanzadas.

Se trata de una base de datos altamente escalable, tanto por la cantidad de datos que puede gestionar como por el número de usuarios simultáneos que puede manejar. Es una base de datos muy robusta y ampliamente utilizada. Tenga en cuenta que puede utilizar MySQL, Oracle o cualquier RDBMS que desee con Airflow. Lo único que tendrá que hacer es cambiar la configuración en airflow.cfg como verá en el siguiente vídeo. Por lo tanto, PostgreSQL debería ser más que suficiente para ejecutar Airflow con el Local Executor. 

Si recuerdas, dije anteriormente que podemos modificar la configuración de Airflow para controlar el número de tareas que queremos ejecutar en paralelo. En realidad, hay tres parámetros en los que tenemos que centrarnos. Para ilustrar la funcionalidad, tomemos el siguiente ejemplo. Así que tenemos dos DAGruns, DAGRun #1 y DAGRun #2, donde ambos DAG tienen cinco tareas c/u, donde tres de ellas se pueden ejecutar en paralelo. Tarea 1, 2 y 3 como se muestra aquí. La tarea 4 depende del éxito de las tres tareas, entonces una vez que la tarea 4 se hace, la tarea 5 se dispara. 

<center><img src="https://i.postimg.cc/23RKZzYz/a753.png"></center>

Muy bien, el primer parámetro es **`parallelism`**. **`Parallelism`** determina cuántas instancias de tareas se pueden ejecutar activamente en paralelo a través de DAGs, dados los recursos disponibles en un momento dado. Se puede considerar como el máximo global de tareas activas. Por ejemplo, si parallelism se establece en tres, entonces sólo se permitirá ejecutar las tareas 1 y 2 de DAGRun # 1 y la tarea 1 de DAGrun # 2. Te estarás preguntando ¿por qué no la tarea 3 de DAGrun # 1? Bueno, en realidad depende de otros dos parámetros que son **`dag_concurrency`** y **`max_active_runs_per_dag`**. El parámetro max_active_runs_per_dag le dice al scheduler que no ejecute más que el número definido de DAGRuns en un momento dado para un DAG específico. Por ejemplo, si establecemos el max_active_runs_per_dag a 1, el DAGRun # 1 se ejecuta primero y una vez que termina, se dispara el DAGRun # 2. Fíjate que este parámetro establece el número máximo de DAGRuns activos por DAG y se aplicará a todos tus DAGs. Si quieres ser más específico, puedes personalizar este número a nivel de DAG estableciendo el parámetro **`max_active_runs`** en la definición del objeto DAG. 

<center><img src="https://i.postimg.cc/WpBNngb4/a754.png"></center>

Volviendo al ejemplo, con parallelism configurado a tres y max_active_runs_per_dag configurado a 1, sólo las tareas 1, 2 y 3 del DAGRun # 1 se ejecutarán en paralelo. El último parámetro que puede afectar a este comportamiento es dag_concurrency. **`dag_concurrency`** determina cuántas instancias de tarea puede programar el scheduler a la vez por DAG. Por ejemplo, si establecemos dag_concurrency en 2 y mantenemos max_active_runs_per_dag y parallelism respectivamente en 1 y 3, entonces sólo se ejecutarán las tareas 1 y 2 del DAGRun # 1. Puede parecer confuso, así que tómate tu tiempo y no dudes en reproducir lo que acabo de decir. Muy bien, ahora que hemos visto cómo configurar el paralelismo en Airflow, vamos a ver lo que significa para el Local Executor.

<center><img src="https://i.postimg.cc/SNcytRJf/a755.png"></center>

El Local Executor implementa dos estrategias. Estableciendo el parámetro parallelism a 0, el Local Executor generará un proceso cada vez que se envíe una tarea al executor y se ejecutará en su propio proceso. Básicamente, tan pronto como se reciba una tarea, se asignará un nuevo proceso a la tarea. Esta estrategia se denomina Paralelismo Ilimitado (Unlimited Parallelism). Lo cual no es exactamente cierto, ya que, depende de los recursos disponibles en tu sistema. La segunda estrategia es Paralelismo Limitado (Limited Parallelism) que se obtiene cuando se establece el parámetro parallelism a un número mayor que 0. En esta estrategia, el Local Executor utilizará una cola (queue) para gestionar el número definido de procesos y las tareas a ejecutar. Como mejor práctica, deberías establecer parallelism al número de cores disponibles en tu máquina menos uno. Especialmente si sus tareas están "ligadas a la CPU" donde la mayor parte de la tarea transcurre en la CPU, en contraste con las tareas ligadas a la I/O que procesan datos desde un disco. Además, nunca se debe establecer el parámetro a 0, ya que, podría hacer inestable su máquina por el potencial número ilimitado de procesos que el executor puede crear. Al final, tienes que hacer tus propias pruebas para saber cuál es el número correcto a establecer de acuerdo con el rendimiento que quieres y obtienes. Bueno, basta de hablar, lo entenderás mejor con el ejemplo práctico.

<center><img src="https://i.postimg.cc/7PGP0m4c/a756.png"></center>